In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
from vqls import *
from utils import *


In [11]:
func_out = {'sigmoid': sigmoid_t,'tanh': tanh_t,'elu': elu_t, 'relu': relu_t, 'sin':sin_m}
func_list = list(func_out.keys())
func_dict = {'sigmoid': .0,'tanh': 1.0,'elu':.12, 'relu':.0, 'sin':2}

pnq = {'3':(3,8),
        '4':(4,16)}
columns=['function', 'nq', 'knot', 'training cost', 'time(s)', 'weights', 'RSS_h', 'RSS_q' ]
tot = {}
for flab, fun in func_out.items():
    for nq, knot in  pnq.values():
        file_path = f"results/results_gvqs_{flab}_{nq}.json"
        df = pd.read_json(file_path).drop(['Condition number', 'norm(yk)', 'training_cost', 'exe_time','RSS_h'], axis =1 )
        tmp = []
        for el in df["in_train_weights"]:
             tmp.append(el[-1])
        df["weights"] = tmp
        df = df.drop(["in_train_weights"], axis = 1)
        tot[f"{flab}{nq}"] = df 

results/results_gvqs_sigmoid_3.json
results/results_gvqs_sigmoid_4.json
results/results_gvqs_tanh_3.json
results/results_gvqs_tanh_4.json
results/results_gvqs_elu_3.json
results/results_gvqs_elu_4.json
results/results_gvqs_relu_3.json
results/results_gvqs_relu_4.json
results/results_gvqs_sin_3.json
results/results_gvqs_sin_4.json


In [12]:
rss=[]
for flab, fun in func_out.items():
    for nq in [3,4]:
        dl = tot[f"{flab}{nq}"]
        rmse_scal = []
        rmse_yc = []
        for i, el in dl.iterrows():
            
            lower = 0.
            upper = 1.

            f_i = func_dict[flab]
            scaled=False    
            label = flab
            n_step = pnq[f"{nq}"][1]
            x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
            xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1
            y = [fun(value,f_i) for value in xx]

            #tck=splrep(x,y,k=1) #coeffs
            norm = np.linalg.norm(y)
            y = y / norm
            matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)

            vqls_circuit = VQLS(matrix,v_norm,nq,opt='COBYLA') 

            c = np.linalg.solve(matrix,vector)

            y_c=np.dot(matrix,c) #classic

            y_fq=[]
            q = vqls_circuit.solution(el['weights']).real
            y_q=np.dot(matrix,q) #hybrid
                            #Quantum

            for a in matrix:
                y_fq.append(vqls_circuit.direct_prod2(el['weights'],a))
            rmse_yc.append(math.sqrt(np.square(np.subtract(y_c,y_fq)).mean()))
            rmse_scal.append(math.sqrt(np.square(np.subtract(y,y_fq)).mean()))
        dl["rmse_scal"] = rmse_scal
        dl["rmse_yc"] = rmse_yc


In [17]:
tot.keys()

dict_keys(['sigmoid3', 'sigmoid4', 'tanh3', 'tanh4', 'elu3', 'elu4', 'relu3', 'relu4', 'sin3', 'sin4'])

In [23]:
dk= tot["sigmoid3"]
#dk['RSS_q'] = dk['RSS_q']/dk['knot']
print(dk.pivot( columns = [], values=['rmse_yc','RSS_q']).sort_values(["rmse_yc"]).to_markdown())

|       |   rmse_yc |    RSS_q |
|:------|----------:|---------:|
| (12,) |  0.183223 | 0.268566 |
| (22,) |  0.18553  | 0.275371 |
| (11,) |  0.185691 | 0.275849 |
| (3,)  |  0.185943 | 0.276599 |
| (23,) |  0.698045 | 3.89813  |
| (6,)  |  0.702102 | 3.94358  |
| (17,) |  0.702309 | 3.9459   |
| (13,) |  0.702592 | 3.94908  |
| (18,) |  0.703671 | 3.96122  |
| (0,)  |  0.703877 | 3.96354  |
| (8,)  |  0.704963 | 3.97578  |
| (10,) |  0.706013 | 3.98764  |
| (9,)  |  0.715782 | 4.09875  |
| (19,) |  0.719192 | 4.1379   |
| (4,)  |  0.719874 | 4.14575  |
| (20,) |  0.719874 | 4.14575  |
| (14,) |  0.721589 | 4.16553  |
| (15,) |  0.725029 | 4.20533  |
| (7,)  |  0.738071 | 4.35799  |
| (5,)  |  1.08453  | 9.40971  |
| (21,) |  1.08523  | 9.42185  |
| (2,)  |  1.08552  | 9.42683  |
| (1,)  |  1.08588  | 9.43312  |
| (16,) |  1.08667  | 9.44684  |
| (24,) |  1.08667  | 9.44684  |


In [45]:
flab = "sigmoid"
nq = 3
dl = tot[f"{flab}{nq}"]
dl["RSS_q"][3]


0.2765988787

In [48]:
rss=[]
res = {}

for flab, fun in func_out.items():
    funres = {}
    # vqs results
    for stp in [20,40]:
        nq = 1
        file_path = f"results/results_vqs_{flab}_{nq}_{stp}.json"
        dl = pd.read_json(file_path).drop(["exe_time","weights", "training_cost", "seed"], axis = 1)

        funres[f"vqs_{stp}"]= {"rmse":np.min(dl["rmse"]), "index": np.argmin(dl["rmse"]), "RSS": dl["RSS_q"][np.argmin(dl["rmse"])] }


    # gvqs results
    for nq in [3,4]:
        dl = tot[f"{flab}{nq}"]
        rmse_scal = []
        rmse_yc = []
        lower = 0.
        upper = 1.
        for i, el in dl.iterrows():
            

            f_i = func_dict[flab]
            scaled=False    
            label = flab
            n_step = pnq[f"{nq}"][1]
            x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
            xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1

            #tck=splrep(x,y,k=1) #coeffs
            matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)

            vqls_circuit = VQLS(matrix,v_norm,nq,opt='COBYLA') 

            c = np.linalg.solve(matrix,vector)

            y_c=np.dot(matrix,c) #classic

            y_fq=[]
            q = vqls_circuit.solution(el['weights']).real
            y_q=np.dot(matrix,q) #hybrid
                            #Quantum

            for a in matrix:
                y_fq.append(vqls_circuit.direct_prod2(el['weights'],a))
            rmse_yc.append(math.sqrt(np.square(np.subtract(y_c,y_fq)).mean()))
        funres[f"gvqs_{nq}"]= {"rmse":np.min(rmse_yc), "index": np.argmin(rmse_yc), "RSS": dl["RSS_q"][np.argmin(rmse_yc)] }
    res[flab] = funres


In [50]:
res["sin"]

{'vqs_20': {'rmse': 0.2898510146, 'index': 4, 'RSS': 3.3605444271},
 'vqs_40': {'rmse': 0.29246201850000003,
  'index': 0,
  'RSS': 6.8427225829000005},
 'gvqs_3': {'rmse': 0.22396438450233014, 'index': 5, 'RSS': 0.4012803642},
 'gvqs_4': {'rmse': 0.31198818624321956,
  'index': 4,
  'RSS': 1.5573860537000002}}